# Design Single Country Nat Cat Bonds

In [1]:
#import generald packages
import numpy as np

#import climada stuff
from climada.hazard import tc_tracks, Centroids, TropCyclone
from climada.entity import LitPop
from climada.util.api_client import Client
from climada.entity.impact_funcs import trop_cyclone, impact_func_set
from climada.engine import ImpactCalc
import functions as fct
import exposures as ex


client = Client()

c:\Users\kaibe\miniforge3\envs\climada_env\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


### Load TC Tracks

In [2]:
basins = ['EP','NA','NI','SI','SP','WP']

storms_basin = fct.init_STORM_tracks(basins=basins, load_haz=True)

----------------------Loading Hazard----------------------


KeyboardInterrupt: 

### Generate Exposure Data 
Generate exposure data using newest available data by myself. Can choose fin_mode myself, reference year, and resolution. 


In [ ]:
#Generate exposure data
country = 480
fin_mode = 'gdp'
res_arc = 30
ref_year = 2020

exp, centrs, applicable_basins = ex.generate_exposure(country, fin_mode, ref_year, res_arc, load_exp=True)

### Generate TC Class

In [ ]:
#filter tc tracks of applicalble basin which are located close to exposure values
buffer = 1.0

storm_basin_sub = fct.filter_tc_tracks(storms_basin, basins_exp_storm, exp, buffer)

ax_sub = storm_basin_sub[basins_exp_storm].plot()
ax_sub.get_legend()._loc = 2

In [ ]:
#define frequency correction factor for STORM dataset
freq_corr_STORM = 1/1000
#generate tc class
tc_storms = fct.init_tc_hazard(storm_basin_sub, basins, freq_corr_STORM)

### Calculate Impact

In [ ]:
#import regional calibrated impact function for TC
# prepare impact calcuation - after Samuel Eberenz
# The iso3n codes need to be consistent with the column “region_id” in the 
# 1. Init impact functions:
impact_func_set = trop_cyclone.ImpfSetTropCyclone()
impf_set = impact_func_set.from_calibrated_regional_ImpfSet()
impf_set.check()

# get mapping: country ISO3n per region:
iso3n_per_region = impf_id_per_region = impact_func_set.get_countries_per_region()[2]
    
code_regions = {'NA1': 1, 'NA2': 2, 'NI': 3, 'OC': 4, 'SI': 5, 'WP1': 6, \
                'WP2': 7, 'WP3': 8, 'WP4': 9, 'ROW': 10}

# match exposure with correspoding impact function
for calibration_region in impf_id_per_region:
    for country_iso3n in iso3n_per_region[calibration_region]:
        exp.gdf.loc[exp.gdf.region_id== country_iso3n, 'if_TC'] = code_regions[calibration_region]
    
# calculate impact
imp = ImpactCalc(exp, impact_func_set, tc_storms[storm_basin_sub]).impact(save_mat=True)


In [ ]:
#have a look at exposure geodataframe
exp.gdf.head()

In [ ]:
#compute exceedance frequency curve
freq_curve = imp.calc_freq_curve()
freq_curve.plot()
imp.aai_agg
imp_ev = imp.at_event